In [1]:
import torch
print(torch.__version__)  # 检查 PyTorch 版本
print(torch.version.cuda)  # 检查 CUDA 版本
print(torch.cuda.is_available())  # 检查 CUDA 是否可用
print(torch.cuda.device_count())  # Should be > 0

2.4.1+cu121
12.1
True
1


In [1]:
import hashlib
import os
import tarfile
import zipfile
import requests

#@save
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'
def download(name, cache_dir=os.path.join('..', 'data')):  #@save
    """下载一个DATA_HUB中的文件，返回本地文件名"""
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # 命中缓存
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname
def download_extract(name, folder=None):  #@save
    """下载并解压zip/tar文件"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():  #@save
    """下载DATA_HUB中的所有文件"""
    for name in DATA_HUB:
        download(name)

In [2]:
DATA_HUB['kaggle_house_train'] = (  #@save
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  #@save
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

In [3]:
import pandas as pd 
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd 
import numpy as np  
train_data = TabularDataset(r'C:\DeepLearning\california-house-prices\train.csv')
# train_data = pd.read_csv(download('kaggle_house_train'))
#模型质量
print(train_data.describe())
id, label = 'Id', 'Sold Price'
#数据预处理
# larger_val_col = ['Lot', 'Total interior livable area',
#                   'Tax assessed value','Annual tax amount',
#                   'Listed Price', 'Last Sold Price']
# for c in larger_val_col + [label]:
#     train_data[c] = np.log(train_data[c]+1)

    
predictor = TabularPredictor(label=label).fit(
    train_data.drop(columns=[id]),num_gpus=1,
)


Loaded data from: C:\DeepLearning\california-house-prices\train.csv | Columns = 41 / 41 | Rows = 47439 -> 47439
No path specified. Models will be saved in: "AutogluonModels\ag-20250130_022418"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          24
Memory Avail:       16.29 GB / 31.12 GB (52.4%)
Disk Space Avail:   678.21 GB / 924.66 GB (73.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'    

                 Id    Sold Price    Year built           Lot     Bathrooms  \
count  47439.000000  4.743900e+04  46394.000000  3.325800e+04  43974.000000   
mean   23719.000000  1.296050e+06   1956.634888  2.353383e+05      2.355642   
std    13694.604047  1.694452e+06    145.802456  1.192507e+07      1.188805   
min        0.000000  1.005000e+05      0.000000  0.000000e+00      0.000000   
25%    11859.500000  5.650000e+05   1946.000000  4.991000e+03      2.000000   
50%    23719.000000  9.600000e+05   1967.000000  6.502000e+03      2.000000   
75%    35578.500000  1.525000e+06   1989.000000  1.045400e+04      3.000000   
max    47438.000000  9.000000e+07   9999.000000  1.897474e+09     24.000000   

       Full bathrooms  Total interior livable area  Total spaces  \
count    39574.000000                 4.491300e+04  46523.000000   
mean         2.094961                 5.774587e+03      1.567117   
std          0.963320                 8.324363e+05      9.011608   
min          1.0

	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['Address', 'Summary', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 'Elementary School', 'Middle School', 'High School', 'Flooring', 'Heating features', 'Cooling features', 'Appliances included', 'Laundry features', 'Parking features']
			CountVectorizer fit with vocabulary size = 10000
		Reducing Vectorizer vocab size from 10000 to 6244 to avoid OOM error
	Stage 4 Generators:
		Fitting DropUniqueFeat

In [5]:
test_data = TabularDataset(r'C:\DeepLearning\california-house-prices\test.csv')
predictions = predictor.predict(test_data.drop(columns=[id]))
submission = pd.DataFrame({'Id': test_data[id], 'Sold Price': predictions})
submission.to_csv('submission.csv', index=False)

Loaded data from: C:\DeepLearning\california-house-prices\test.csv | Columns = 40 / 40 | Rows = 31626 -> 31626


: 